# Notebook with the code used to create the slide "A reproducibility detour" for the talk at the 2nd International Workshop on Quantum Computing Software (in conjunction with Supercomputing ’21)

Parameters from https://arxiv.org/abs/2107.00677

In [1]:
import networkx as nx
import numpy as np
from functools import partial

from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer

from qiskit.providers.aer import AerSimulator
from qiskit.algorithms.minimum_eigen_solvers.qaoa import QAOAAnsatz
from qiskit.algorithms.optimizers import SPSA


from QAOAKit.utils import brute_force

In [2]:
p = 11
beta = [
        0.6564387949836022,
        0.5633471282117171,
        0.5163014280756274,
        0.5036682504396424,
        0.4818229959468761,
        0.45601255796623946,
        0.421434626428442,
        0.3707732712794268,
        0.27645017426067187,
        0.20094698054318946,
        0.10725070156301893
      ]
gamma = [
        0.257287401304126,
        0.5280405724278872,
        0.5917709544691275,
        0.6396914913796506,
        0.6772056138057753,
        0.7020614534590051,
        0.7370242568081657,
        0.7753074973682114,
        0.8835587304543469,
        1.046970255431072,
        1.1152377164688672
      ]

# Generate a graph
n = 10
graph = nx.random_regular_graph(3,n, seed=42)

In [3]:
def cut(x, G):
    cut = 0
    for u,v in G.edges():
        if x[u] != x[v]:
            cut += 1
    return cut

In [4]:
# Compute the weight matrix from the graph
w = nx.adjacency_matrix(graph)

# Formulate the problem as quadratic program
problem = QuadraticProgram()
_ = [problem.binary_var(f"x{i}") for i in range(n)]  # create n binary variables
linear = w.dot(np.ones(n))
quadratic = -w
problem.maximize(linear=linear, quadratic=quadratic)
obj = partial(cut, G=graph)

In [5]:
def angles_to_qiskit_ordering(beta, gamma):
    return np.concatenate(
        [[g, b] for g, b in zip(gamma, beta)]
    )

# Get circuit
C, offset = problem.to_ising()
ansatz = QAOAAnsatz(C, p).decompose()
qc = ansatz.bind_parameters(angles_to_qiskit_ordering(beta, gamma))
qc.measure_all()

# run circuit
backend = AerSimulator()
counts = backend.run(qc).result().get_counts()

exp = 0
total_count = 0
for k, v in counts.items():
    exp += obj(k[::-1]) * v
    total_count += v
    
exp /= total_count

true_opt = brute_force(obj, n)[0]
print(f"Expected cut: {exp}\nTrue optimum: {true_opt}\nRatio: {exp / true_opt}")

Expected cut: 0.755859375
True optimum: 13
Ratio: 0.05814302884615385


In [6]:
# Compute the weight matrix from the graph
w = nx.adjacency_matrix(graph, nodelist=range(10))

# Formulate the problem as quadratic program
problem = QuadraticProgram()
_ = [problem.binary_var(f"x{i}") for i in range(n)]  # create n binary variables
linear = w.dot(np.ones(n))
quadratic = -w
problem.maximize(linear=linear, quadratic=quadratic)
obj = partial(cut, G=graph)

In [7]:
def angles_to_qiskit_ordering(beta, gamma):
    return np.concatenate(
        [[g, b] for g, b in zip(gamma, beta)]
    )

# Get circuit
C, offset = problem.to_ising()
ansatz = QAOAAnsatz(C, p).decompose()
qc = ansatz.bind_parameters(angles_to_qiskit_ordering(beta, gamma))
qc.measure_all()

# run circuit
backend = AerSimulator()
counts = backend.run(qc).result().get_counts()

exp = 0
total_count = 0
for k, v in counts.items():
    exp += obj(k[::-1]) * v
    total_count += v
    
exp /= total_count

true_opt = brute_force(obj, n)[0]
print(f"Expected cut: {exp}\nTrue optimum: {true_opt}\nRatio: {exp / true_opt}")

Expected cut: 0.359375
True optimum: 13
Ratio: 0.027644230769230768


In [8]:
def angles_to_qiskit_ordering(beta, gamma):
    return np.concatenate(
        [[-g, b] for g, b in zip(gamma, beta)]
    )

# Get circuit
C, offset = problem.to_ising()
ansatz = QAOAAnsatz(C, p).decompose()
qc = ansatz.bind_parameters(angles_to_qiskit_ordering(beta, gamma))
qc.measure_all()

# run circuit
backend = AerSimulator()
counts = backend.run(qc).result().get_counts()

exp = 0
total_count = 0
for k, v in counts.items():
    exp += obj(k[::-1]) * v
    total_count += v
    
exp /= total_count

true_opt = brute_force(obj, n)[0]
print(f"Expected cut: {exp}\nTrue optimum: {true_opt}\nRatio: {exp / true_opt}")

Expected cut: 12.767578125
True optimum: 13
Ratio: 0.9821213942307693
